In [1]:
import pandas as pd
import openpyxl
from IPython.display import display
import xlsxwriter
import numpy as np

In [2]:
df = pd.read_excel(r"C:\Users\zhangbon\Desktop\temp\PLM-MBOM全量导出 - 20250430-匹配增量数据.xlsx")


In [3]:
df.dtypes

元数据标准名称                object
工厂                      int64
父级物料编码                 object
项目号                     int64
子级物料编码                 object
计数器                   float64
更改号自                   object
更改号至                   object
物料供应标识                 object
单位                     object
数量                    float64
生产仓储地点                float64
有效期自           datetime64[ns]
有效期至                   object
Unnamed: 14            object
备注1                    object
备注2                    object
dtype: object

In [4]:
# 先将字段转换为字符串
df['有效期自'] = df['有效期自'].astype(str)
df['有效期至'] = df['有效期至'].astype(str)

# 处理9999年的特殊情况
df.loc[df['有效期自'].str.contains('9999'), '有效期自'] = '9999/12/31'
df.loc[df['有效期至'].str.contains('9999'), '有效期至'] = '9999/12/31'

# 转换其他日期
df.loc[~df['有效期自'].str.contains('9999'), '有效期自'] = pd.to_datetime(
    df.loc[~df['有效期自'].str.contains('9999'), '有效期自']
).dt.strftime('%Y/%m/%d')

df.loc[~df['有效期至'].str.contains('9999'), '有效期至'] = pd.to_datetime(
    df.loc[~df['有效期至'].str.contains('9999'), '有效期至']
).dt.strftime('%Y/%m/%d')

print('转换完成')

# 检查结果
print("\n转换后的前几行数据：")
print(df[['有效期自', '有效期至']].head())

转换完成

转换后的前几行数据：
         有效期自        有效期至
0  2018/05/27  9999/12/31
1  2018/05/27  9999/12/31
2  2018/05/27  9999/12/31
3  2018/05/27  9999/12/31
4  2018/05/27  9999/12/31


In [5]:
import datetime

# 获取当前日期
current_date = datetime.datetime.now().strftime('%Y/%m/%d')
current_date_dt = pd.to_datetime(current_date)

# 创建状态标记列
df['状态标记'] = '已失效'  # 默认状态

# 处理9999年的特殊情况
max_date = pd.Timestamp.max  # pandas支持的最大日期

# 将日期转换为datetime进行比较，特殊处理9999年
def convert_to_datetime(date_str):
    if '9999' in str(date_str):
        return max_date
    return pd.to_datetime(date_str)

# 安全转换日期
df['有效期自_dt'] = df['有效期自'].apply(convert_to_datetime)
df['有效期至_dt'] = df['有效期至'].apply(convert_to_datetime)

# 标记状态
df.loc[(df['有效期自_dt'] <= current_date_dt) & (df['有效期至_dt'] >= current_date_dt), '状态标记'] = '现在生效'
df.loc[df['有效期自_dt'] > current_date_dt, '状态标记'] = '未来生效'

# 删除临时的datetime列
df = df.drop(['有效期自_dt', '有效期至_dt'], axis=1)

# 显示结果统计
print("状态标记统计：")
print(df['状态标记'].value_counts())

# 显示部分示例数据
print("\n示例数据：")
print(df[['有效期自', '有效期至', '状态标记']].head(10))

状态标记统计：
状态标记
现在生效    482196
未来生效     12069
Name: count, dtype: int64

示例数据：
         有效期自        有效期至  状态标记
0  2018/05/27  9999/12/31  现在生效
1  2018/05/27  9999/12/31  现在生效
2  2018/05/27  9999/12/31  现在生效
3  2018/05/27  9999/12/31  现在生效
4  2018/05/27  9999/12/31  现在生效
5  2016/10/11  9999/12/31  现在生效
6  2016/10/11  9999/12/31  现在生效
7  2016/10/11  9999/12/31  现在生效
8  2016/10/11  9999/12/31  现在生效
9  2025/02/20  9999/12/31  现在生效


In [6]:
#筛选出生效和未来生效的
df_valid = df[df['状态标记'].isin(['现在生效', '未来生效'])]

# # 保存结果到Excel
# df_valid.to_excel(r"C:\Users\zhangbon\Desktop\临时活\MBOM\PLM-MBOM全量导出现在和未来.xlsx", sheet_name='Sheet1', index=False)


In [7]:
# #导入用于筛选是否冻结的物料的文件
# df_freeze = pd.read_excel(r"C:\Users\PC\Desktop\temp\用于筛选BOM父级是否冻结.xlsx")

In [8]:
# df_freeze

In [9]:
# #筛选出df_freeze当中已经被冻结的物料 
# df_freeze_ = df_freeze[df_freeze['是否冻结'] == '已冻结']
# #在df_valid中筛选出未冻结的物料
# df_valid_not_freeze = df_valid[~df_valid['父级物料编码'].isin(df_freeze_['物料编码'])]
df_valid_not_freeze = df_valid.fillna(' ')

In [10]:
# df_valid_not_freeze

In [11]:
# df_valid_not_freeze.dtypes
# df_valid_not_freeze['物料供应标识']=df_valid_not_freeze['物料供应标识'].fillna('')
# #筛选出备注为Make的数据
# df_valid_not_freeze = df_valid_not_freeze[df_valid_not_freeze['备注'] == 'Make']
# len(df_valid_not_freeze)
# df_valid_not_freeze.columns

In [12]:
# 首先创建一个明确的副本
df_valid_not_freeze = df_valid_not_freeze.copy()

# 创建一个字典来存储每个父级物料编码在不同工厂的数据
comparison_dict = {}

# 遍历数据，建立比较基础
for _, row in df_valid_not_freeze.iterrows():
    parent = row['父级物料编码']
    factory = row['工厂']
    if parent not in comparison_dict:
        comparison_dict[parent] = {}
    if factory not in comparison_dict[parent]:
        comparison_dict[parent][factory] = []
    comparison_dict[parent][factory].append({
        '子级物料编码': row['子级物料编码'],
        '项目号': row['项目号'],
        '单位': row['单位'],
        '数量': row['数量'],
        '物料供应标识': row['物料供应标识'],
        '状态标记': row['状态标记']
    })

# 创建新的标记列
df_valid_not_freeze.loc[:, '子级物料不同'] = False
df_valid_not_freeze.loc[:, '项目号不同'] = False
df_valid_not_freeze.loc[:, '单位不同'] = False
df_valid_not_freeze.loc[:, '数量不同'] = False
df_valid_not_freeze.loc[:, '物料供应标识不同'] = False
df_valid_not_freeze.loc[:, '状态标记不同'] = False
df_valid_not_freeze.loc[:, '差异工厂'] = ''
df_valid_not_freeze.loc[:, '差异说明'] = ''  # 新增差异说明列

# 比较并标记差异
for idx, row in df_valid_not_freeze.iterrows():
    parent = row['父级物料编码']
    current_factory = row['工厂']
    
    if len(comparison_dict[parent]) <= 1:
        continue
    
    different_factories = []
    differences = []  # 用于收集差异说明
    
    for other_factory, other_data in comparison_dict[parent].items():
        if other_factory != current_factory:
            current_child = {
                '子级物料编码': row['子级物料编码'],
                '项目号': row['项目号'],
                '单位': row['单位'],
                '数量': row['数量'],
                '物料供应标识': row['物料供应标识'],
                '状态标记': row['状态标记']
            }
            
            found_match = False
            for other_item in other_data:
                if other_item['子级物料编码'] == current_child['子级物料编码']:
                    found_match = True
                    if other_item['项目号'] != current_child['项目号']:
                        df_valid_not_freeze.loc[idx, '项目号不同'] = True
                        differences.append(f"工厂{other_factory}项目号为{other_item['项目号']}")
                    if other_item['单位'] != current_child['单位']:
                        df_valid_not_freeze.loc[idx, '单位不同'] = True
                        differences.append(f"工厂{other_factory}单位为{other_item['单位']}")
                    if other_item['数量'] != current_child['数量']:
                        df_valid_not_freeze.loc[idx, '数量不同'] = True
                        differences.append(f"工厂{other_factory}数量为{other_item['数量']}")
                    if other_item['物料供应标识'] != current_child['物料供应标识']:
                        df_valid_not_freeze.loc[idx, '物料供应标识不同'] = True
                        differences.append(f"工厂{other_factory}物料供应标识为{other_item['物料供应标识']}")
                    if other_item['状态标记'] != current_child['状态标记']:
                        df_valid_not_freeze.loc[idx, '状态标记不同'] = True
                        differences.append(f"工厂{other_factory}状态标记为{other_item['状态标记']}")
                    if other_factory not in different_factories:
                        different_factories.append(str(other_factory))
            
            if not found_match:
                df_valid_not_freeze.loc[idx, '子级物料不同'] = True
                differences.append(f"工厂{other_factory}未找到该子级物料")
                if other_factory not in different_factories:
                    different_factories.append(str(other_factory))
    
    if different_factories:
        df_valid_not_freeze.loc[idx, '差异工厂'] = ','.join(different_factories)
        df_valid_not_freeze.loc[idx, '差异说明'] = '; '.join(differences)  # 合并所有差异说明

# 添加一个"是否存在差异"列
df_valid_not_freeze.loc[:, '是否存在差异'] = (
    df_valid_not_freeze['子级物料不同'] |
    df_valid_not_freeze['项目号不同'] |
    df_valid_not_freeze['单位不同'] |
    df_valid_not_freeze['数量不同'] |
    df_valid_not_freeze['物料供应标识不同'] |
    df_valid_not_freeze['状态标记不同']
)

# 显示有差异的记录
print("差异统计：")
print("存在差异的记录总数:", df_valid_not_freeze['是否存在差异'].sum())
print("子级物料不同的记录数:", df_valid_not_freeze['子级物料不同'].sum())
print("项目号不同的记录数:", df_valid_not_freeze['项目号不同'].sum())
print("单位不同的记录数:", df_valid_not_freeze['单位不同'].sum())
print("数量不同的记录数:", df_valid_not_freeze['数量不同'].sum())
print("物料供应标识不同的记录数:", df_valid_not_freeze['物料供应标识不同'].sum())
print("状态标记不同的记录数:", df_valid_not_freeze['状态标记不同'].sum())

# 显示部分示例数据
print("\n示例数据（包含差异的记录）：")
columns_to_show = ['工厂', '父级物料编码', '子级物料编码', '项目号', '单位', '数量', '物料供应标识', '状态标记',
                   '是否存在差异', '差异说明', '差异工厂']
print(df_valid_not_freeze[df_valid_not_freeze['是否存在差异']][columns_to_show].head())

# 保存结果到Excel文件
df_valid_not_freeze.to_excel(r"C:\Users\zhangbon\Desktop\temp\BOM差异对比结果.xlsx", index=False)

差异统计：
存在差异的记录总数: 34380
子级物料不同的记录数: 1543
项目号不同的记录数: 560
单位不同的记录数: 0
数量不同的记录数: 242
物料供应标识不同的记录数: 32057
状态标记不同的记录数: 150

示例数据（包含差异的记录）：
        工厂         父级物料编码         子级物料编码   项目号  单位     数量 物料供应标识  状态标记  \
1610  1005  1101000503720  1506060100002     2  KG  0.325         现在生效   
1613  1001  1101000503720  1506060300503  5001  KG  0.300         现在生效   
1655  1003  1109000902050  1109000902520    48  PC  1.000         现在生效   
1668  1003  1109000902050  1109000902370    51  PC  2.000         现在生效   
1671  1003  1109000902050  1504010200012    40  PC  2.000         现在生效   

      是否存在差异            差异说明  差异工厂  
1610    True  工厂1001未找到该子级物料  1001  
1613    True  工厂1005未找到该子级物料  1005  
1655    True  工厂1002未找到该子级物料  1002  
1668    True  工厂1002未找到该子级物料  1002  
1671    True    工厂1002数量为4.0  1002  
